In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"

srcs_ids = []
srcs_ids = [55034 ,61491 ,64360 ,73354 ,76243 ,92867 ,104078 ,106745 ,226975 ,267514 ,588567 ,599887 ,]  # WDs ALL

In [ ]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed

from loguru import logger
logger.enable("vasca")

rg = Region()
rg.load_from_fits(region_fname)
rc = rg.get_region_catalog()
rc.cross_match_cds(query_table="simbad")
rc.cross_match_cds()
rc.write_to_fits(region_fname.replace(".fits","_cat.fits", overwrite=True))

2023-10-06 15:12:35.747 | DEBUG    | vasca.tables:load_from_fits:328 - Loading file with name './vasca_pipeline/ALL_10-800/region_ALL_10-800.fits'
2023-10-06 15:12:35.759 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_fields'
2023-10-06 15:12:35.785 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_filters'
2023-10-06 15:12:35.796 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_visits'
2023-10-06 15:12:35.814 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_detections'
2023-10-06 15:12:36.421 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_detections'
2023-10-06 15:12:36.636 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_sources'
2023-10-06 15:12:36.885 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_coadd_sources'
2023-10-06 15:12:37.525 | DEBUG    | vasca.tables:load_from_fits:339 - Loading table 'tt_src_id_map'
2023-10-06 15:12:37.561 | DEBUG    | vasca.region: